In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
import warnings
import xgboost as xgb

warnings.filterwarnings("ignore")

In [2]:
train = pd.read_parquet("../data/train.parquet")


In [2]:
test = pd.read_parquet("../data/test.parquet")

In [4]:
# Features for balancing
features = train.columns.to_list()
features = features[1:]

In [3]:
features_test = ["P_2", "target", "B_1"]

## Preparing test set for submission

In [4]:

def iv_woe(data, feature, target, test,bins=10, show_woe=False, show_iv= False):
    #Empty Dataframe
    newDF,woeDF = pd.DataFrame(), pd.DataFrame()
    #Extract Column Names
    cols = [feature]
    #Run WOE and IV on all the independent variables
    for ivars in cols:
        if (data[ivars].dtype.kind in 'bifc') and (len(np.unique(data[ivars]))>10):
            binned_x = pd.qcut(data[ivars], bins,  duplicates='drop')
            binned_t = pd.qcut(test[ivars], bins, duplicates='drop')
            # data[ivars]= binned_x
            # data[ivars] = data[ivars].astype(str)
            test[ivars] = binned_t
            test[ivars] = test[ivars].astype(str)
            d0 = pd.DataFrame({'x': binned_x, 'y': data[target]})
            d0 = d0.astype({"x": str})
        else:
            d0 = pd.DataFrame({'x': data[ivars], 'y': data[target]})
            # data[ivars] = data[ivars]
            test[ivars] = test[ivars]
    
    d = d0.groupby("x", as_index=False, dropna=False).agg({"y": ["count", "sum"]})
    d.columns = ['Cutoff', 'N', 'Events']
    
    d.insert(loc=0, column='Variable', value=ivars)

    d['% of Events'] = np.maximum(d['Events'], 0.5) / d['Events'].sum()
    d['Non-Events'] = d['N'] - d['Events']
    d['% of Non-Events'] = np.maximum(d['Non-Events'], 0.5) / d['Non-Events'].sum()
    d['WoE'] = np.log(d['% of Non-Events']/d['% of Events'])
    d['IV'] = d['WoE'] * (d['% of Non-Events']-d['% of Events'])

        
    temp =pd.DataFrame({"Variable" : [ivars], "IV" : [d['IV'].sum()]}, columns = ["Variable", "IV"])
    newDF=pd.concat([newDF,temp], axis=0)
    woeDF=pd.concat([woeDF,d], axis=0)
    return newDF, woeDF

In [5]:
variaveis = train[features_test].columns
#df1 = {}
l = 0
for col in variaveis:
    if col == "target" or col == "customer_ID" or col == "S_2": continue
    else:
        print("WoE e IV: {}".format(col))
        iv, df = iv_woe(train, col, "target", test)
        # df1[l] = df
        # df1[l]["nome"] = "WOE_"+col
        print(l)
        l = l + 1
        #print(tabulate(df, headers="keys"))
        #print("IV score: {:.2f}".format(iv))
        print("\n")
        i = 0
        for i in range(0,len(df)):
            test.loc[test[col] == df.iloc[i,1], col] = df.iloc[i,7]
        #del df, iv
        gc.collect()

WoE e IV: P_2
0


WoE e IV: B_1
1




In [6]:
df.head(16)

,Variable,Cutoff,N,Events,% of Events,Non-Events,% of Non-Events,WoE,IV
0,B_1,"(-7.590000000000001, 0.004]",553146,38988,0.028296,514158,0.123787,1.475844,0.140929
1,B_1,"(0.004, 0.00733]",553145,39210,0.028457,513935,0.123733,1.469732,0.140030
2,B_1,"(0.00733, 0.0106]",553145,39913,0.028967,513232,0.123564,1.450593,0.137221
3,B_1,"(0.0106, 0.0199]",553145,46232,0.033553,506913,0.122042,1.291234,0.114260
4,B_1,"(0.0199, 0.0313]",553145,54956,0.039885,498189,0.119942,1.101014,0.088144
5,B_1,"(0.0313, 0.0503]",553145,83679,0.060731,469466,0.113027,0.621175,0.032485
6,B_1,"(0.0503, 0.0968]",553145,169387,0.122934,383758,0.092392,-0.285607,0.008723
7,B_1,"(0.0968, 0.188]",553145,272960,0.198103,280185,0.067456,-1.077308,0.140747
8,B_1,"(0.188, 0.396]",553145,292436,0.212238,260709,0.062767,-1.218274,0.182096
9,B_1,"(0.396, 1.324]",553145,340108,0.246836,213037,0.051290,-1.571230,0.307248


In [6]:
test["B_1"]

0           0.010728
1           0.011026
2           0.016390
3           0.021672
4           0.015923
              ...   
11363757    0.008638
11363758    0.009636
11363759    0.004420
11363760    0.009529
11363761    0.000346
Name: B_1, Length: 11363762, dtype: float32

In [10]:
test["B_1"].value_counts()

(-7.058000000000001, 0.00435]    1136377
(0.412, 1.324]                   1136377
(0.0079, 0.0124]                 1136376
(0.0124, 0.0216]                 1136376
(0.0216, 0.0334]                 1136376
(0.198, 0.412]                   1136376
(0.0334, 0.0541]                 1136376
(0.0541, 0.103]                  1136376
(0.00435, 0.0079]                1136376
(0.103, 0.198]                   1136376
Name: B_1, dtype: int64

_____________________________________________

In [ ]:
test.to_parquet("../data/test_balanced[70:-1].parquet")

In [ ]:
model = xgb.Booster()

In [ ]:
model.load_model("../models/XGB_V_fold4.xgb")

In [ ]:
d_test_matrix = xgb.DMatrix(test[FEATURES])

In [ ]:
predictions = model.predict(d_test_matrix)

In [3]:
# CALCULATE SIZE OF EACH SEPARATE TEST PART
def get_rows(customers, test, NUM_PARTS = 4, verbose = ''):
    chunk = len(customers)//NUM_PARTS
    if verbose != '':
        print(f'We will process {verbose} data as {NUM_PARTS} separate parts.')
        print(f'There will be {chunk} customers in each part (except the last part).')
        print('Below are number of rows in each part:')
    rows = []

    for k in range(NUM_PARTS):
        if k==NUM_PARTS-1: cc = customers[k*chunk:]
        else: cc = customers[k*chunk:(k+1)*chunk]
        s = test.loc[test.customer_ID.isin(cc)].shape[0]
        rows.append(s)
    if verbose != '': print( rows )
    return rows,chunk

# COMPUTE SIZE OF 4 PARTS FOR TEST DATA
NUM_PARTS = 4
TEST_PATH = '../data/test.parquet'

print(f'Reading test data...')
test = read_file(path = TEST_PATH, usecols = ['customer_ID','S_2'])
customers = test[['customer_ID']].drop_duplicates().sort_index().values.flatten()
rows,num_cust = get_rows(customers, test[['customer_ID']], NUM_PARTS = NUM_PARTS, verbose = 'test')

Reading test data...


NameError: name 'read_file' is not defined

In [ ]:
# INFER TEST DATA IN PARTS
skip_rows = 0
skip_cust = 0
test_preds = []

for k in range(NUM_PARTS):
    
    # READ PART OF TEST DATA
    print(f'\nReading test data...')
    test = read_file(path = TEST_PATH)
    test = test.iloc[skip_rows:skip_rows+rows[k]]
    skip_rows += rows[k]
    print(f'=> Test part {k+1} has shape', test.shape )
    
    # PROCESS AND FEATURE ENGINEER PART OF TEST DATA
    test = process_and_feature_engineer(test)
    if k==NUM_PARTS-1: test = test.loc[customers[skip_cust:]]
    else: test = test.loc[customers[skip_cust:skip_cust+num_cust]]
    skip_cust += num_cust
    
    # TEST DATA FOR XGB
    X_test = test[FEATURES]
    dtest = xgb.DMatrix(data=X_test)
    test = test[['P_2_mean']] # reduce memory
    del X_test
    gc.collect()

    # INFER XGB MODELS ON TEST DATA
    model = xgb.Booster()
    model.load_model(f'XGB_v{VER}_fold0.xgb')
    preds = model.predict(dtest)
    for f in range(1,FOLDS):
        model.load_model(f'XGB_v{VER}_fold{f}.xgb')
        preds += model.predict(dtest)
    preds /= FOLDS
    test_preds.append(preds)

    # CLEAN MEMORY
    del dtest, model
    _ = gc.collect()

In [ ]:
# WRITE SUBMISSION FILE
test_preds = np.concatenate(test_preds)
test = cudf.DataFrame(index=customers,data={'prediction':test_preds})
sub = cudf.read_csv('../input/amex-default-prediction/sample_submission.csv')[['customer_ID']]
sub['customer_ID_hash'] = sub['customer_ID'].str[-16:].str.hex_to_int().astype('int64')
sub = sub.set_index('customer_ID_hash')
sub = sub.merge(test[['prediction']], left_index=True, right_index=True, how='left')
sub = sub.reset_index(drop=True)

# DISPLAY PREDICTIONS
sub.to_csv(f'submission_xgb_v{VER}.csv',index=False)
print('Submission file shape is', sub.shape )
sub.head()

In [ ]:
# PLOT PREDICTIONS
plt.hist(sub.to_pandas().prediction, bins=100)
plt.title('Test Predictions')
plt.show()